Initializing Table

In [1]:
import tabula
import pandas as pd

#top,left,bottom,right of box
box =  [1.97, .11, 2.79, 7.77]
options = {'names': ["Last Raced", "Pgm", "Horse Name (Jockey)", "Wgt", "M/E", "PP", "Start", "1/4", "3/8", "Str", "Fin", "Odds", "Comments"]}
for i in range(0,len(box)):
    box[i] *= 72
    
test_area = [135.246,7.92,206,559.44]
df = tabula.read_pdf("equibaseFile.pdf", pages = 1, area = [test_area])

In [2]:
data = df[0]
#data = data.drop(index = 0)
data

,Last Raced,Unnamed: 0,Pgm,Horse Name (Jockey),Wgt M/E,PP,Start,1/4,3/8,Str,Fin,Odds,Comments
0,2 1,NaN,NaN,NaN,NaN,NaN,NaN,Head,Head,2,Nose,NaN,NaN
1,2Jul22 MTH,NaN,4.0,"Wild Mule (Jimenez, Albin)",118 L,4.0,3.0,2,2,2,1,0.90*,"battled back, got nod"
2,6 1,NaN,NaN,NaN,NaN,NaN,NaN,3,4,Head,3/4,NaN,NaN
3,2Jul22 MTH,NaN,6.0,"Rol Again Dancer (Rodriguez, Angel)",118 L,5.0,2.0,3,3,1,2,1.50,"vied 3w, led, missed"
4,2 1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Head,3/4,NaN,NaN
5,30Jun22 DEL,NaN,1.0,"Bright Forecast (Ferrer, Jose)",118 L,1.0,5.0,5,5,4,3,4.70,"off slowly, rallied"
6,1 1,NaN,NaN,NaN,NaN,NaN,NaN,4,2 1/2,NaN,2,NaN,NaN
7,22Jul22 MTH,NaN,2.0,"Running Right by U (Vargas, Jr., Jorge)",118 L,2.0,4.0,4,4,5,4,14.40,"inside, no rally"
8,5 7,NaN,NaN,NaN,NaN,NaN,NaN,1/2,1/2,3,NaN,NaN,NaN
9,30Jul22 MTH,NaN,3.0,"Run Happy Pappy (Torres, Jomar)",118 L b,3.0,1.0,1,1,3,5,14.10,"dueled inside, faded"


Splitting Wgt & M/E

In [3]:
data[['Wgt', 'M/E']] = data['Wgt M/E'].str.split(' ', 1,expand = True)
data

,Last Raced,Unnamed: 0,Pgm,Horse Name (Jockey),Wgt M/E,PP,Start,1/4,3/8,Str,Fin,Odds,Comments,Wgt,M/E
0,2 1,NaN,NaN,NaN,NaN,NaN,NaN,Head,Head,2,Nose,NaN,NaN,NaN,NaN
1,2Jul22 MTH,NaN,4.0,"Wild Mule (Jimenez, Albin)",118 L,4.0,3.0,2,2,2,1,0.90*,"battled back, got nod",118,L
2,6 1,NaN,NaN,NaN,NaN,NaN,NaN,3,4,Head,3/4,NaN,NaN,NaN,NaN
3,2Jul22 MTH,NaN,6.0,"Rol Again Dancer (Rodriguez, Angel)",118 L,5.0,2.0,3,3,1,2,1.50,"vied 3w, led, missed",118,L
4,2 1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Head,3/4,NaN,NaN,NaN,NaN
5,30Jun22 DEL,NaN,1.0,"Bright Forecast (Ferrer, Jose)",118 L,1.0,5.0,5,5,4,3,4.70,"off slowly, rallied",118,L
6,1 1,NaN,NaN,NaN,NaN,NaN,NaN,4,2 1/2,NaN,2,NaN,NaN,NaN,NaN
7,22Jul22 MTH,NaN,2.0,"Running Right by U (Vargas, Jr., Jorge)",118 L,2.0,4.0,4,4,5,4,14.40,"inside, no rally",118,L
8,5 7,NaN,NaN,NaN,NaN,NaN,NaN,1/2,1/2,3,NaN,NaN,NaN,NaN,NaN
9,30Jul22 MTH,NaN,3.0,"Run Happy Pappy (Torres, Jomar)",118 L b,3.0,1.0,1,1,3,5,14.10,"dueled inside, faded",118,L b


Functions

In [4]:
from PyPDF2 import PdfReader
def number_horses(file, #Pdf file
                 page_num): #Page number of pdf
    reader = PdfReader(file)
    page = reader.pages[page_num - 1]
    text = page.extract_text()
    tokenized_text = text.split()
    
    start_count_bool = False
    count = 0
    for i in range(0,len(tokenized_text) - 1):
        if(tokenized_text[i] == "Trainers:"):
            start_count_bool = True
        elif(tokenized_text[i] == "Owners:"):
            start_count_bool = False
        elif(start_count_bool and tokenized_text[i] == '-'):
            count += 1
    return count

In [5]:
#If top headers are not found
def tinker_top_bounds(table_df, #Dataframe of imported table from pdf 
                      table_loc_df, #Dataframe of locations of tables on pdf
                      table_num): #Table we are parsing for (1 = top, 2 = bottom)
    #Setting header value to look for
    if(table_num == 1):
        target_header = 'Last Raced'
    else:
        target_header = 'Pgm'
        
    col_names = list(table_df.columns.values)
    
    #Loop until top bound is right
    bound_num = 10 #Increased bounds by 10, both positive and negative to find the right headers. Ex: 10, -10, 20, -20, 30...
    while(col_names[0] != target_header or bound_num > 200):
        
        top_bound = table_loc_df.iloc[table_num - 1,2]
        top_bound += bound_num
        test_area = [
            top_bound,
            table_loc_df.iloc[table_num - 1,3],
            table_loc_df.iloc[table_num - 1,4],
            table_loc_df.iloc[table_num - 1,5]
        ]
        scan_df = tabula.read_pdf("equibaseFile.pdf", pages = 2, area = [test_area])
        table_df = scan_df[0]
        col_names = list(table_df.columns.values)

        #Change bound_num
        if(bound_num < 0): #If number is in negative cycle
            bound_num *= -1
            bound_num += 10
        else:
            bound_num *= -1
            
    return table_df

In [20]:
def tinker_bottom_bounds(table_df, #Dataframe of imported table from pdf 
                         table_loc_df, #Dataframe of locations of tables on pdf
                         table_num): #Table we are parsing for (1 = top, 2 = bottom)

    num_horses = number_horses("equibaseFile.pdf", 2)
    num_target_rows = num_horses * 2
    num_rows = len(table_df.index)
    
    bound_num = 10 #Increment bound by 10
    while(num_rows < num_target_rows or bound_num > 200):
        bottom_bound = table_loc_df.iloc[table_num - 1,2]
        bottom_bound += bound_num
        test_area = [
            table_loc_df.iloc[table_num - 1,2],
            table_loc_df.iloc[table_num - 1,3],
            bottom_bound,
            table_loc_df.iloc[table_num - 1,5]
        ]
        try:
            scan_df = tabula.read_pdf("equibaseFile.pdf", pages = 2, area = [test_area])
            table_df = scan_df[0]
        except:
            print("Couldn't read a table, moving on from {} bottom_bound".format(bottom_bound))
            
        num_rows = len(table_df.index)

        bound_num += 10
    
    #Decrement bound by 10
    while(num_rows > num_target_rows or bound_num < -200):
        bottom_bound = table_loc_df.iloc[table_num - 1,2]
        bottom_bound += bound_num
        test_area = [
            table_loc_df.iloc[table_num - 1,2],
            table_loc_df.iloc[table_num - 1,3],
            bottom_bound,
            table_loc_df.iloc[table_num - 1,5]
        ]
        
        try:
            scan_df = tabula.read_pdf("equibaseFile.pdf", pages = 2, area = [test_area])
            table_df = scan_df[0] #2nd loop
        except:
            print("Couldn't read a table, moving on from {} bottom_bound".format(bottom_bound))
        num_rows = len(table_df.index)

        bound_num -= 10
        
    return table_df

Getting Table Areas

In [7]:
#Function Definitions
from pathlib import Path
from typing import Iterable, Any

from pdfminer.high_level import extract_pages
from pdfminer.layout import LTTextContainer


def extract_to_df(pages: Any):
    cols = ["Page","Text", "x_1", "y_1", "x_2", "y_2"]
    words = ["Last Raced", "Fractional Times", "Past Performance Running Line Preview", "Fin", "Trainers"]
    text_loc_df = pd.DataFrame(columns = cols)
    
    page_counter = 0
    for page_layout in pages:
        page_counter += 1
        word_counter = 0
        for element in page_layout:
            if isinstance(element, LTTextContainer):
                element_text = element.get_text()
                for word in words:
                    if(element_text.__contains__(word)):
                        loc_nums = []
                        for num in element.bbox:
                            loc_nums.append(num) #Adding binding box numbers
                        text_loc_df.loc[len(text_loc_df.index)] = [page_counter,
                                                                     word,
                                                                     loc_nums[0], 
                                                                     loc_nums[1], 
                                                                     loc_nums[2], 
                                                                     loc_nums[3]]
    return text_loc_df

In [8]:
#Extracting pages for table locations

pages = extract_pages("equibaseFile.pdf")
text_loc_df = extract_to_df(pages)
text_loc_df

The PDF <_io.BufferedReader name='equibaseFile.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding. Use the check_extractable if you want to raise an error in this case


,Page,Text,x_1,y_1,x_2,y_2
0,1,Last Raced,9.920,593.754,57.893,650.314
1,1,Fin,438.062,593.754,455.958,650.314
2,1,Fractional Times,9.920,558.454,154.659,584.454
3,1,Fin,231.019,576.454,301.226,584.454
4,1,Past Performance Running Line Preview,209.385,324.169,373.977,392.229
...,...,...,...,...,...,...
72,11,Fin,438.062,593.754,458.634,650.314
73,11,Fin,231.019,576.454,301.226,584.454
74,11,Past Performance Running Line Preview,209.385,283.669,372.981,351.729
75,11,Fin,389.760,283.669,410.332,340.229


In [9]:
#Data Cleaning

#Selecting only page 1

text_loc_df = text_loc_df.loc[text_loc_df['Page'] == 1] 


#Saving only last instance of fin; Lowest y_1

fin_df = text_loc_df.loc[text_loc_df['Text'] == "Fin"] #df with only Fin
fin_df = fin_df.sort_values(by=['y_1']) #Sort by y_1 descending
fin_row = fin_df.iloc[0] #Save 1st row
text_loc_df = text_loc_df.loc[text_loc_df['Text'] != "Fin"] #Delete Fin rows
text_loc_df = text_loc_df.append(fin_row) #Append right Fin row


#Saving only last instance of Pgm Horse Name; Lowest y_1

#pgm_df = text_loc_df.loc[text_loc_df['Text'] == "Pgm Horse Name"] #df with only Fin
#pgm_df = pgm_df.sort_values(by=['y_1']) #Sort by y_1 descending
#pgm_row = pgm_df.iloc[0] #Save 1st row
#text_loc_df = text_loc_df.loc[text_loc_df['Text'] != "Pgm Horse Name"] #Delete Fin rows
#text_loc_df = text_loc_df.append(pgm_row) #Append right Fin row

#Resort table by index
text_loc_df = text_loc_df.sort_index()
text_loc_df

C:\Users\jonho\AppData\Local\Temp/ipykernel_32/727711778.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  text_loc_df = text_loc_df.append(fin_row) #Append right Fin row


,Page,Text,x_1,y_1,x_2,y_2
0,1,Last Raced,9.920,593.754,57.893,650.314
2,1,Fractional Times,9.920,558.454,154.659,584.454
4,1,Past Performance Running Line Preview,209.385,324.169,373.977,392.229
5,1,Fin,389.760,324.169,407.656,380.729
6,1,Trainers,9.920,303.535,390.629,311.535


In [10]:
#Creating table location dataframe
df = text_loc_df
cols = ["Page","Table",'Top','Left','Bottom','Right']
table_loc_df = pd.DataFrame(columns = cols)

#For each page
page = 1
#Top Table

left = 7.92
right = 559.44

y_1 = df.loc[df['Text'] == "Last Raced",'y_1'].tolist()[0] #Getting y_1 value
top = (((729 - y_1) * 11)/792) *72

y_1 = df.loc[df['Text'] == "Fractional Times",'y_1'].tolist()[0]
bottom = (((729 - y_1) * 11)/792 + .50) *72

#Adding Table 1 to df
table_loc_df.loc[len(table_loc_df.index)] =[page,1,top,left,bottom,right]

#Bottom Table

left = 126
right = 424.08

print(y_1)
y_1 = df.loc[df['Text'] == "Past Performance Running Line Preview",'y_1'].tolist()[0]
print(y_1)
top = (((729 - y_1) * 11)/792) * 72
print(top)

y_1 = df.loc[df['Text'] == "Trainers",'y_1'].tolist()[0]
bottom = ((((729 - y_1) * 11)/792 + .75)) * 72

#Adding Table 2 to df
table_loc_df.loc[len(table_loc_df.index)] =[page,2,top,left,bottom,right]


558.4540000000001
324.169
404.8310000000001


In [11]:
table_loc_df

,Page,Table,Top,Left,Bottom,Right
0,1.0,1.0,135.246,7.92,206.546,559.44
1,1.0,2.0,404.831,126.00,479.465,424.08


In [12]:
test_area = [
    table_loc_df.iloc[1,2],
    table_loc_df.iloc[1,3],
    table_loc_df.iloc[1,4],
    table_loc_df.iloc[1,5]
]
df = tabula.read_pdf("equibaseFile.pdf", pages = 2, area = [test_area])
df
bottom_table = df[0]

In [13]:
bottom_table

,Pgm,Horse Name,Start,1/4,1/2,3/4,Str,Fin
0,NaN,NaN,NaN,3 1/4,2 3/4,1 3/4,2 1/2,Nec
1,5.0,Kingdom On Paws,7.0,6,6,5,3,1
2,NaN,NaN,NaN,3 1/2,3 3/4,Head,1/2,Nec
3,4.0,Office Etiquette,6.0,7,7,2,1,2
4,NaN,NaN,NaN,Head,1/2,Head,1/2,3 1/4
5,6.0,Tap the Candy,3.0,2,1,1,2,3
6,NaN,NaN,NaN,1 3/4,1 1/2,1 1/2,4,4 1/2
7,3.0,Ninth of April,4.0,5,4,4,4,4
8,NaN,NaN,NaN,1 3/4,1/2,1/2,6,7 1/4
9,8.0,Strasbourg,5.0,4,3,3,5,5


In [14]:
new_table = tinker_top_bounds(bottom_table,table_loc_df,2)
new_table

,Pgm,Horse Name,Start,1/4,1/2,3/4,Str,Fin
0,NaN,NaN,NaN,3 1/4,2 3/4,1 3/4,2 1/2,Nec
1,5.0,Kingdom On Paws,7.0,6,6,5,3,1
2,NaN,NaN,NaN,3 1/2,3 3/4,Head,1/2,Nec
3,4.0,Office Etiquette,6.0,7,7,2,1,2
4,NaN,NaN,NaN,Head,1/2,Head,1/2,3 1/4
5,6.0,Tap the Candy,3.0,2,1,1,2,3
6,NaN,NaN,NaN,1 3/4,1 1/2,1 1/2,4,4 1/2
7,3.0,Ninth of April,4.0,5,4,4,4,4
8,NaN,NaN,NaN,1 3/4,1/2,1/2,6,7 1/4
9,8.0,Strasbourg,5.0,4,3,3,5,5


In [21]:
tinker_bottom_bounds(bottom_table,table_loc_df,2)

Couldn't read a table, moving on from 414.8310000000001 bottom_bound


,Pgm,Horse Name,Start,1/4,1/2,3/4,Str,Fin
0,NaN,NaN,NaN,3 1/4,2 3/4,1 3/4,2 1/2,Nec
1,5.0,Kingdom On Paws,7.0,6,6,5,3,1
2,NaN,NaN,NaN,3 1/2,3 3/4,Head,1/2,Nec
3,4.0,Office Etiquette,6.0,7,7,2,1,2
4,NaN,NaN,NaN,Head,1/2,Head,1/2,3 1/4
5,6.0,Tap the Candy,3.0,2,1,1,2,3
6,NaN,NaN,NaN,1 3/4,1 1/2,1 1/2,4,4 1/2
7,3.0,Ninth of April,4.0,5,4,4,4,4
8,NaN,NaN,NaN,1 3/4,1/2,1/2,6,7 1/4
9,8.0,Strasbourg,5.0,4,3,3,5,5
